# A14 - DCT studies.
Stough, DIP
In conjunction with the [paper activity](https://docs.google.com/document/d/1fZOkp1fE5Q0Z64qmcuveyZX86h0T9McICnlsdeqWha8/edit?usp=sharing).

In [14]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

from waveletUtil import (makeStandardMatrix,
                         makeHaarMatrix,
                         makeDCTMatrix,
                         makeRandomBasis,
                         makeKLTBasis)

In [15]:
def arrInfo(arr):
    return arr.shape, arr.min(), arr.max(), arr.dtype

In [16]:
I = plt.imread('shamokin.png')[...,:3] #remove alpha chan
arrInfo(I)


((512, 512, 3), 0.003921569, 1.0, dtype('float32'))

In [17]:
plt.figure(figsize=(4,4))
plt.imshow(I);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
DCT = makeDCTMatrix(225)
I = plt.imread('index.png')[...,:3] #remove alpha chan
print(arrInfo(DCT))
print(arrInfo(I))
T = np.zeros(I.shape[:2] + (3,))
for chan in range(3):
    T[...,chan] = np.matmul(DCT, np.matmul(I[..., chan], DCT.transpose()))
print(arrInfo(T))
plt.figure()
plt.imshow(DCT)


((225, 225), -0.09428090415820634, 0.09428090415820634, dtype('float64'))
((225, 225, 3), 0.0, 1.0, dtype('float32'))
((225, 225, 3), -27.96842197770711, 175.1670942238466, dtype('float64'))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
plt.figure()
plt.hist(T[...,0].ravel(),np.arange(-2,2,.01))
print(arrInfo(T))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

((225, 225, 3), -27.96842197770711, 175.1670942238466, dtype('float64'))


In [20]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


def lossly(keep=50):
    Tc = np.zeros(T.shape)
    DCT = makeDCTMatrix(225)
    Tr = np.zeros(T.shape)

    for chan in range(3):
        cutoff = np.percentile(np.abs(T[...,chan].ravel()), keep)
        mask = (np.abs(T[...,0]) >= cutoff)
        Tc[mask,chan] = T[mask,chan]
    
    
  
    for chan in range(3):
        Tr[...,chan] = np.matmul(DCT.T, np.matmul(Tc[..., chan], DCT))

    print(arrInfo(Tr))
    ax[0].cla()
    ax[0].imshow(np.clip(Tr,0,1))
    ax[1].cla()
    ax[1].imshow(np.clip(I-Tr,0,1))
    print((~mask).sum()) # Falses
    print((mask).sum()) # Trues

In [21]:
#DCT.shape
##print(arrInfo(Tc))
#print(cutoff)
#print((~mask).sum()) # Falses
#print((mask).sum()) # Trues

In [22]:
f, ax = plt.subplots(1,2, figsize=(10,3),sharex=True,sharey=True)

interact(lossly, keep=(0,100))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=50, description='keep'), Output()), _dom_classes=('widget-interact',))

<function __main__.lossly(keep=50)>

In [23]:


def DCTfun(block, keep):
    Q = np.zeros((8,8)).astype(int)
    Q[0] = [16,11,10,16,24,40,51,61]
    Q[1] = [12,12,14,19,26,58,60,55]
    Q[2] = [14,13,16,24,40,57,69,56]
    Q[3] = [14,17,22,29,51,87,80,62]
    Q[4] = [18,22,37,56,68,109,103,77]
    Q[5] = [24,35,55,64,81,104,113,92]
    Q[6] = [49,64,78,87,103,121,120,101]
    Q[7] = [72,92,95,98,112,100,103,99]
    
    
    
    DCT = makeDCTMatrix(block.shape[0])    
    I = block
    T = np.zeros(I.shape[:2] + (3,))
    B = np.zeros(block.shape)
    for chan in range(3):
        T[...,chan] = np.matmul(DCT, np.matmul(I[..., chan], DCT.transpose())) # getting cooefficents
        
        B[...,chan] = np.round(T[...,chan]/Q) #quntization TODO Q*keep as factor add bool button to change between the Q and the percentile
        B[...,chan] = B[...,chan]*Q
    Tr = np.zeros(T.shape)
    Tc = np.zeros(T.shape)
    Bc = np.zeros(T.shape)
    
    
    

    for chan in range(3):
        Bc[...,chan] = B[...,chan]
        
        cutoff = np.percentile(np.abs(T[...,chan].ravel()), keep)
        mask = (np.abs(T[...,0]) >= cutoff)
        Tc[mask,chan] = T[mask,chan]
    
    if(keep==0):
        Tc = Bc  
    
  
    for chan in range(3):
        Tr[...,chan] = np.matmul(DCT.T, np.matmul(Tc[..., chan], DCT)) #rebuilding
 
    return Tr

In [32]:
from skimage.util import *
Ir = (plt.imread('eye.png')[...,:3])[:-1,:-1,:] #remove alpha chan
Ir = Ir
def Jpeg(drop,useQ=False):
    '''
    sizelist = []
    for i in range(1, Ir.shape[0] + 1):
        if Ir.shape[0] % i == 0:
            sizelist.append(i)
    blocksize= sizelist[blocksize]
    '''
    blocksize =8
    
    DCT = makeDCTMatrix(blocksize)
    I = (plt.imread('eye.png')[...,:3])[:-1,:-1,:] #remove alpha chan
    
    I = I *255
    print(arrInfo(I))
    
    
    block_shape = (blocksize,blocksize,3)

    view = view_as_blocks(I, block_shape=block_shape)
    view = np.squeeze(view)
    blockView = view.reshape([view.shape[0]*view.shape[1]] + list(view.shape[2:]))
    newImgBlock1 = np.zeros(blockView.shape)
    
    
    for i, block in enumerate(blockView):
        #bT = np.mean(block, axis=(0,1)) 
        newImgBlock1[i][:] = DCTfun(block,useQ)# -128 because wiki says so

    IComp = montage(newImgBlock1, grid_shape=[view.shape[0], view.shape[1]], multichannel=True)
    IComp = IComp -IComp.min()
    IComp = IComp/IComp.max()
    plt.imshow(IComp)
    print(blocksize)
    print(drop)
    print(arrInfo(IComp))

In [33]:
plt.figure()
interact(Jpeg, drop=(0,100),useQ=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=50, description='drop'), Checkbox(value=True, description='useQ'), Outpu…

<function __main__.Jpeg(drop, useQ=False)>

In [4]:
Q = np.zeros((8,8)).astype(int)
Q[0] = [16,11,10,16,24,40,51,61]
Q[1] = [12,12,14,19,26,58,60,55]
Q[2] = [14,13,16,24,40,57,69,56]
Q[3] = [14,17,22,29,51,87,80,62]
Q[4] = [18,22,37,56,68,109,103,77]
Q[5] = [24,35,55,64,81,104,113,92]
Q[6] = [49,64,78,87,103,121,120,101]
Q[7] = [72,92,95,98,112,100,103,99]

In [13]:
I = plt.imread("shamokin.png")[...,:3]
CT = np.zeros(I.shape)

H = makeDCTMatrix(8)    

for c in range(3):
    CT[...,c] = np.mutmul(H, np.matmul(I, np.matmul(I[...,c], H.transpose())))
    


AttributeError: module 'numpy' has no attribute 'mutmul'

In [10]:
plt.figure()
plt.imshow(T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …